In [1]:
import pyspark
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
os.chdir(parent_dir)

In [ ]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder 
    .master("local[8]")
    .appName("delta")
    .config("spark.driver.memory", "13g")
    .config("spark.jars.packages",
            "io.delta:delta-core_2.12:3.3.2")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/06/17 21:28:44 WARN Utils: Your hostname, quero-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.15.151 instead (on interface wlp2s0)
25/06/17 21:28:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/eric-yoshida/.ivy2/cache
The jars for the packages stored in: /home/eric-yoshida/.ivy2/jars


:: loading settings :: url = jar:file:/home/eric-yoshida/pyspark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dea6b4ed-154e-4ea3-a3b9-1aa2528c206d;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.2 in central
	found io.delta#delta-storage;3.3.2 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 115ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.2 from central in [default]
	io.delta#delta-storage;3.3.2 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: r

In [4]:
data_path = "./data/processed/"

united_df = spark.read.json(data_path + "united_df.json")

In [5]:
united_df.limit(10).toPandas()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,0005089549,None,5.0,So sorry I didn't purchase this years ago when...,"11 9, 2012",A2M1CU2IRZG0K9,Terri,VHS Tape,Amazing!,1352419200,True,None
1,0005089549,None,5.0,Believe me when I tell you that you will recei...,"12 30, 2011",AFTUJYISOFHY6,Melissa D. Abercrombie,VHS Tape,Great Gospel VHS of the Cathedrals!,1325203200,True,None
2,000503860X,None,5.0,"I have seen X live many times, both in the ear...","04 21, 2005",A3JVF9Y53BEOGC,Anthony Thompson,DVD,A great document of a great band,1114041600,True,11
3,000503860X,None,5.0,"I was so excited for this! Finally, a live co...","04 6, 2005",A12VPEOEZS1KTC,JadeRain,DVD,YES!! X LIVE!!,1112745600,True,5
4,000503860X,None,5.0,X is one of the best punk bands ever. I don't ...,"12 3, 2010",ATLZNVLYKP9AZ,T. Fisher,DVD,X have still got it,1291334400,True,5
5,000503860X,None,5.0,I've loved X since I first saw them here in Sa...,"10 17, 2007",A3TNYNA2360NPA,Stanley C. Sargent,DVD,A Revival to match the early shows by a great ...,1192579200,True,None
6,000503860X,None,5.0,I must admit I was hesitant to purchase this D...,"11 8, 2005",A2LUL6PRTXE7SE,PSM/Bokor,DVD,Old punks still rule...,1131408000,True,17
7,0005419263,None,5.0,The little ones love this,"04 12, 2016",A2CFV9UPFTTM10,SuzieQ,Audio CD,Love it,1460419200,True,None
8,0005419263,None,3.0,Good songs. The DVD is a little boring.,"07 28, 2014",A3139J3877Y61F,SingingButterfly,Audio CD,Three Stars,1406505600,True,None
9,0005419263,None,5.0,The DVD came in great condition and provided l...,"12 21, 2010",A2PANT8U0OJNT4,Cutegirlmomma,DVD,Blast from the past,1292889600,True,None


In [6]:
delta_path = "./data/delta/united_data"

united_df.write.format("delta").mode("overwrite").option("overwriteSchema", "True").save(delta_path)

25/06/17 21:29:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
